# Allegro Web API

In [1]:
import hashlib
import base64
import time
import numpy as np

from suds.client import Client, WebFault

In [2]:
api_key = 'xxx'
api_key_sandbox = 'xxx'
login = "xxx"
password = "xxx"

In [3]:
class AllegroWebAPI:
    API_URL_sandbox = 'https://webapi.allegro.pl.webapisandbox.pl/service.php?wsdl'
    API_URL = 'https://webapi.allegro.pl/service.php?wsdl'
    country_code = 1 # 1 dla Polski, 56 dla Czech, 209 dla Ukrainy
 
    def __init__(self, login, password, api_key):
        self.login = login
        self.password_enc = base64.b64encode(hashlib.sha256(password.encode('utf-8')).digest()).decode()
        self.api_key = api_key
        self.client = Client(self.API_URL)
        self.service = self.client.service
        self.version_key = self.get_version_by_country_code()
        self.item_id_list = []
        self.item_dict = {}
 
    def get_version_by_country_code(self):
        systems = self.service.doQueryAllSysStatus(**{
            'countryId': self.country_code,
            'webapiKey':self.api_key
        })[0]
 
        for sys in systems:
            if sys['countryId'] == self.country_code:
                return sys['verKey']
 
    def sign_in_enc(self):
        try:
            self.session_id = self.service.doLoginEnc(
                userLogin= self.login,
                userHashPassword = self.password_enc,
                countryCode = self.country_code,
                webapiKey = self.api_key,
                localVersion = self.version_key
            )
        except WebFault as e:
            print e
            
    def get_categories_tree(self):
        self.categories_tree = self.service.doGetCatsData(self.country_code, self.version_key, self.api_key)
    
    def get_books(self, filter_value_min=30, filter_value_max=31, result_size=200):
        self.item_last_cost = filter_value_min
        
        for cost in np.arange(filter_value_min, filter_value_max, 0.25):
            
            cost2 = cost + 0.249
            if cost2 > filter_value_max:
                cost2 = filter_value_max
            self.item_last_cost = cost + 0.25
            
            table = {'price': {'filterValueRange': [cost, cost2]},
                     'category': {'filterValueId': '91459'}
                    }
            filtr_query = api.client.factory.create('ArrayOfFilteroptionstype')

            for i in range(len(table)):
                filtr = api.client.factory.create('FilterOptionsType')
                filtr.filterId = table.keys()[i]
                if 'filterValueId' in table.values()[i].keys():
                    filtrAOS = api.client.factory.create('ArrayOfString')
                    filtrAOS.item = table.values()[i].values()[0]
                    filtr.filterValueId = filtrAOS
                if 'filterValueRange' in table.values()[i].keys():
                    filtrRVT = api.client.factory.create('RangeValueType')
                    filtrRVT.rangeValueMin = table.values()[i].values()[0][0]
                    filtrRVT.rangeValueMax = table.values()[i].values()[0][1]
                    filtr.filterValueRange = filtrRVT
                filtr_query.item.append(filtr)

            item_list = api.service.doGetItemsList(api.api_key, api.country_code, filtr_query, resultSize = result_size)
            
            print time.strftime("%A, %H:%M:%S"), cost, cost + 0.249

            for item in item_list.itemsList.item:
                item_id = item.itemId
                if item_id not in self.item_dict.keys():
                    self.item_id_list.append(item_id)
                    full_item = api.service.doShowItemInfoExt(api.session_id.sessionHandlePart, item_id, 
                                                              getDesc = 1).itemListInfoExt
                    self.item_dict[str(item_id)] = {'name': full_item.itName, 'description': full_item.itDescription}
    
    def get_books_until_done(self, filter_value_min=30, filter_value_max=31, result_size=200):
        try:
            api.get_books(filter_value_min, filter_value_max, result_size)
        except WebFault as e:
            print 'Probujemy po bledzie'
            api.sign_in_enc()
            api.get_books_until_done(api.item_last_cost, filter_value_max, result_size)

In [4]:
api = AllegroWebAPI(login, password, api_key)
api.sign_in_enc()

In [5]:
api.get_books_until_done(25, 55)

Sunday, 11:30:53 25.0 25.249


No handlers could be found for logger "suds.client"


Probujemy po bledzie
Sunday, 11:31:18 25.25 25.499
Sunday, 11:31:38 25.5 25.749
Sunday, 11:32:32 25.75 25.999
Sunday, 11:33:54 26.0 26.249
Sunday, 11:35:21 26.25 26.499
Sunday, 11:36:34 26.5 26.749
Sunday, 11:37:39 26.75 26.999
Sunday, 11:39:02 27.0 27.249
Sunday, 11:40:30 27.25 27.499
Sunday, 11:41:56 27.5 27.749
Sunday, 11:43:19 27.75 27.999
Sunday, 11:44:41 28.0 28.249
Probujemy po bledzie
Sunday, 11:45:02 28.25 28.499
Sunday, 11:45:34 28.5 28.749
Sunday, 11:46:05 28.75 28.999
Sunday, 11:46:51 29.0 29.249
Sunday, 11:48:05 29.25 29.499
Sunday, 11:49:03 29.5 29.749
Sunday, 11:49:52 29.75 29.999
Probujemy po bledzie
Sunday, 11:50:43 30.0 30.249
Sunday, 11:51:55 30.25 30.499
Sunday, 11:53:07 30.5 30.749
Sunday, 11:53:26 30.75 30.999
Sunday, 11:54:40 31.0 31.249
Sunday, 11:55:56 31.25 31.499
Sunday, 11:56:46 31.5 31.749
Sunday, 11:58:00 31.75 31.999
Sunday, 11:59:04 32.0 32.249
Sunday, 12:00:18 32.25 32.499
Sunday, 12:00:54 32.5 32.749
Sunday, 12:01:51 32.75 32.999
Sunday, 12:02:58 33.0 

In [6]:
import pickle

with open('list_dict.pickle', 'wb') as handle:
    b = pickle.dump(api.item_dict, handle)

In [12]:
len(api.item_id_list)

19851

# Dodatkowy kod dla drzewa kategorii

In [ ]:
api.categories_tree

In [ ]:
category_name = "Książki i Komiksy"
api.get_categories_tree()
for category in api.categories_tree.catsList.item:
    if category["catName"] == category_name:
        category_id = category["catId"]
        break
    

In [ ]:
api.service.doGetCategoryPath(api.session_id.sessionHandlePart, 91459)